In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.read_csv('../input/titanic/train.csv',index_col=0)
train.head()

In [ ]:
train.describe()

In [ ]:
# relleno de datos de edad con KNN
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
train[['Age']] = imputer.fit_transform(train[['Age']])
train.describe()

In [ ]:
#crear nuevas caracteristicas
# Tamaño de la familia
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1

# Extraer el título del nombre
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Categorizar Fare y Age
train['FareBin'] = pd.qcut(train['Fare'], 4, labels=False)
train['AgeBin'] = pd.cut(train['Age'].fillna(-0.5), bins=[-1, 0, 5, 12, 18, 35, 60, 100], labels=False)
train.head()

In [ ]:
# Eliminar columnas innecesarias
train = train.drop(columns=['Name','Ticket', 'Fare', 'Age', 'Cabin', 'SibSp', 'Parch']) 


In [ ]:
# Convertir las columnas categóricas a variables dummy
train = pd.get_dummies(train, columns=['Sex', 'Embarked', 'Title'], drop_first=True)

In [ ]:
train.isnull().sum()

### entrenamiento del modelo

In [ ]:
# definir la variable a predir y las variables X
y = train.Survived
X = train.drop('Survived',axis=1)

In [ ]:
# Dividir el dataset en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Crear una instancia del modelo de árbol de decisión
rf_clf = RandomForestClassifier(random_state=0, n_estimators=100)

# Entrenar el modelo con los datos de entrenamiento
rf_clf.fit(X_train, y_train)

In [ ]:
# Hacer predicciones con los datos de prueba
y_pred = rf_clf.predict(X_test)

# Evaluar el rendimiento del modelo
from sklearn.metrics import accuracy_score, classification_report
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

### mejoras del modelo

In [ ]:
# Definir el grid de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Configurar el GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

In [ ]:
# Obtener el mejor modelo
best_rf_clf = grid_search.best_estimator_

In [ ]:
# Evaluar el rendimiento del modelo con los mejores parámetros
y_pred = best_rf_clf.predict(X_test)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# entrenamiento final y resultados

In [ ]:
# Entrenar el modelo con todos los datos de entrenamiento
best_rf_clf.fit(X, y)

In [ ]:
# carga y transformacion de los datos test.csv
test = pd.read_csv('../input/titanic/test.csv',index_col=0)

In [ ]:
# relleno de datos de edad con KNN
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
test[['Age']] = imputer.fit_transform(test[['Age']])
test.describe()

In [ ]:
#test['Fare'].fillna(test['Fare'].median(), inplace=True)

test['Fare'].fillna(test['Fare'].median(), inplace=True)

In [ ]:
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['FareBin'] = pd.qcut(test['Fare'], 4, labels=False)
test['AgeBin'] = pd.cut(test['Age'].fillna(-0.5), bins=[-1, 0, 5, 12, 18, 35, 60, 100], labels=False)

test.head()

In [ ]:
test = test.drop(columns=['Name', 'Fare', 'Age', 'Cabin', 'Ticket', 'SibSp', 'Parch'])

In [ ]:
# Convertir las columnas categóricas a variables dummy
test = pd.get_dummies(test, columns=['Sex', 'Embarked', 'Title'], drop_first=True)

In [ ]:
test.head()

In [ ]:
# Cambiar el nombre de la columna (suponiendo que don y dona hacen referencia a cargos similares)
test = test.rename(columns={'Title_Dona': 'Title_Don'})
test.head()

In [ ]:
# Alinear las columnas del conjunto de test con las del conjunto de entrenamiento
missing_cols = set(X.columns) - set(test.columns)
for c in missing_cols:
    test[c] = 0

# Asegurarse de que el orden de las columnas sea el mismo
test = test[X.columns]

In [ ]:
# Asegurarse de que las columnas en test coinciden con las de entrenamiento
test = test.reindex(columns=X.columns, fill_value=0)

In [ ]:
# Realizar predicciones sobre los datos no etiquetados
test_predictions = best_rf_clf.predict(test)

In [ ]:
print(test_predictions)

In [ ]:
test['Survived'] = test_predictions

In [ ]:
test.head()

In [ ]:
# Mostrar el DataFrame con PassengerId y Prediction
test = test.reset_index()
result_df = test[['PassengerId', 'Survived']]

# guardar el archivo en csv
result_df.to_csv('submission2.csv', index=False)

# Verificar el resultado
print(result_df)

In [ ]:
# lectura del Output
output2 = pd.read_csv('/kaggle/working/submission2.csv')
output2.Survived

# Contar la cantidad de sobrevivientes y no sobrevivientes
counts = output2['Survived'].value_counts()

print("Cantidad de sobrevivientes:", counts.get(1, 0))  # Imprime la cantidad de sobrevivientes
print("Cantidad de no sobrevivientes:", counts.get(0, 0))  # Imprime la cantidad de no sobrevivientes